In [3]:
import numpy as np
from scipy.optimize import linprog

# Problem data
num_facilities = 3
num_customers = 4
num_scenarios = 5

# Fixed costs for facilities
fixed_costs = np.array([100, 150, 200])

# Transportation costs from facilities to customers (3 facilities x 4 customers)
transportation_costs = np.array([[10, 20, 30, 40],
                                 [15, 25, 35, 45],
                                 [20, 30, 40, 50]])

# Capacity of each facility
capacities = np.array([100, 120, 150])

# Demand scenarios (5 scenarios x 4 customers)
demand_scenarios = np.array([[20, 30, 40, 50],
                             [25, 35, 45, 55],
                             [15, 25, 35, 45],
                             [30, 40, 50, 60],
                             [10, 20, 30, 40]])

# Penalty parameter for the algorithm
rho = 1.0

# Number of iterations
max_iterations = 100

# Convergence tolerance
tolerance = 1e-4


In [4]:
# Initialize decision variables and dual variables
x = np.zeros((num_scenarios, num_facilities, num_customers))
y = np.zeros((num_scenarios, num_facilities))  # Binary decision for opening facilities
lambda_x = np.zeros((num_scenarios, num_facilities, num_customers))
lambda_y = np.zeros((num_scenarios, num_facilities))

# Helper function to solve the subproblem for each scenario
def solve_subproblem(demand, fixed_costs, transportation_costs, capacities, lambda_x, lambda_y, rho, x_avg, y_avg):
    num_facilities, num_customers = transportation_costs.shape

    # Objective function coefficients
    c = np.concatenate([transportation_costs.flatten(), fixed_costs])
    
    # Inequality constraints (capacity constraints and demand constraints)
    A_ub = []
    b_ub = []

    # Capacity constraints
    for i in range(num_facilities):
        constraint = np.zeros(num_facilities * num_customers + num_facilities)
        constraint[i * num_customers: (i + 1) * num_customers] = 1
        A_ub.append(constraint)
        b_ub.append(capacities[i])
    
    # Demand constraints
    for j in range(num_customers):
        constraint = np.zeros(num_facilities * num_customers + num_facilities)
        constraint[j::num_customers] = -1
        A_ub.append(constraint)
        b_ub.append(-demand[j])

    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)

    # Variable bounds
    bounds = [(0, None) for _ in range(num_facilities * num_customers)] + [(0, 1) for _ in range(num_facilities)]

    # Adjust objective function coefficients with dual variables and penalty terms
    adjusted_c = np.concatenate([transportation_costs.flatten() + lambda_x.flatten() + rho * (x_avg.flatten() - x.flatten()), 
                                 fixed_costs + lambda_y + rho * (y_avg - y)])

    # Solve the linear programming problem
    result = linprog(adjusted_c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    
    # Extract solution
    x_sol = result.x[:num_facilities * num_customers].reshape((num_facilities, num_customers))
    y_sol = result.x[num_facilities * num_customers:]
    
    return x_sol, y_sol

# Progressive Hedging Algorithm
for iteration in range(max_iterations):
    x_old = x.copy()
    y_old = y.copy()
    
    # Solve subproblems for each scenario
    for s in range(num_scenarios):
        x[s], y[s] = solve_subproblem(demand_scenarios[s], fixed_costs, transportation_costs, capacities, lambda_x[s], lambda_y[s], rho, x.mean(axis=0), y.mean(axis=0))
    
    # Compute averages
    x_avg = np.mean(x, axis=0)
    y_avg = np.mean(y, axis=0)
    
    # Update dual variables
    for s in range(num_scenarios):
        lambda_x[s] += rho * (x[s] - x_avg)
        lambda_y[s] += rho * (y[s] - y_avg)
    
    # Check for convergence
    if np.max(np.abs(x - x_old)) < tolerance and np.max(np.abs(y - y_old)) < tolerance:
        print(f"Converged in {iteration + 1} iterations")
        break

# Display final decision variables
print("Facility opening decisions (y):")
print(np.round(y_avg))
print("Transportation decisions (x):")
print(np.round(x_avg))


ValueError: operands could not be broadcast together with shapes (12,) (60,) 